# 生成文档相关的统计信息和表征

In [1]:
import pickle
import json
import re
from tqdm import tqdm
from pprint import pprint
from gensim import corpora
from collections import defaultdict

## 生成corpus

In [2]:
# 导入所有的document
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/robust04_data/docset.pkl', 'rb') as f:
    docset = pickle.load(f)

In [6]:
from gensim.parsing.preprocessing import STOPWORDS

In [16]:
# 文本清洗函数
def get_corpus_element(texts):
    clean = lambda t: re.sub('[,?;*!%^&_+():\[\]{}`~@#$=+\\|/<>.\'\"\d]', ' ', t.replace('"', ' ').replace('/', ' ').replace('\\', ' ').replace("'", ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('-', ' ').replace('.', '').replace('&hyph;', ' ').replace('&blank;', ' ').strip().lower())
    preprocessed_text = clean(texts)
    text_list = [word for word in preprocessed_text.lower().split() if (word not in STOPWORDS) and len(word) > 1]
    # remove words that appear only once
    frequency = defaultdict(int)
    for token in text_list: frequency[token] += 1
    clean_texts = [token for token in text_list if frequency[token] > 1]
    return clean_texts

In [19]:
from tqdm import tqdm
corpus = []
for doc in tqdm(docset):
    text = docset[doc]['title'] + docset[doc]['abstractText']
    corpus.append(get_corpus_element(text))
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/robust04_data/corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)

100%|██████████| 528155/528155 [02:32<00:00, 3453.18it/s]


In [20]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/robust04_data/corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)
dictionary = corpora.Dictionary(corpus)
print(dictionary)

Dictionary(231448 unique tokens: ['according', 'age', 'agency', 'andov', 'approval']...)


In [21]:
corpus_bow = [dictionary.doc2bow(document) for document in tqdm(corpus)]

100%|██████████| 528155/528155 [00:47<00:00, 11031.87it/s]


In [23]:
dictionary.save('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/robust04_data/corpus_dictionary.dic')  # 保存下来

In [10]:
# # 直接从文件中导入词库即可
# dictionary = corpora.Dictionary().load('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/robust04_data/corpus_dictionary.dic')
# print(dictionary)

Dictionary(231698 unique tokens: ['according', 'age', 'agency', 'also', 'among']...)


## bicut统计数据获取

In [26]:
import torch as t
import numpy as np

In [30]:
stats_for_bicut = {}
for doc in tqdm(docset):
    stats_for_bicut[doc] = []
    text = docset[doc]['title'] + docset[doc]['abstractText']
    clean_text = get_corpus_element(text)
    stats_for_bicut[doc].append(len(clean_text))
    stats_for_bicut[doc].append(len(np.unique(clean_text)))
    stats_for_bicut[doc].append(dictionary.doc2bow(clean_text))

100%|██████████| 528155/528155 [03:44<00:00, 2352.68it/s]


In [31]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bicut_stats.pkl', 'wb') as f:
    pickle.dump(stats_for_bicut, f)

In [32]:
indices = t.LongTensor([1, 4, 7])
values = t.FloatTensor([1, 1, 1])
t.sparse_coo_tensor(indices.unsqueeze(0), values, (30,)).to_dense()

tensor([0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [35]:
# 取出一个文档用于测试效果
test_texts = docset['FBIS3-11058']['title'] + docset['FBIS3-11058']['abstractText']
test_document = get_corpus_element(test_texts)
print(test_document)
test_vec = dictionary.doc2bow(test_document)
print(test_vec)

['labor', 'regional', 'development', 'regional', 'development', 'projects', 'railway', 'construction', 'labor', 'leaders', 'fear', 'plans', 'privatize', 'state', 'owned', 'enfe', 'national', 'railway', 'enterprise', 'ende', 'national', 'enterprise', 'projects', 'cochabamba', 'construction', 'labor', 'leaders', 'government', 'misicuni', 'multiple', 'project', 'government', 'approximately', 'million', 'cost', 'international', 'institutions', 'construction', 'labor', 'leaders', 'project', 'fear', 'government', 'implementation', 'misicuni', 'project', 'government', 'privatize', 'ende', 'government', 'state', 'owned', 'enterprises', 'international', 'institutions', 'enterprises', 'ende', 'said', 'leader', 'misicuni', 'multiple', 'project', 'implementation', 'misicuni', 'project', 'cochabamba', 'projects', 'misicuni', 'projects', 'according', 'railway', 'labor', 'leaders', 'government', 'financing', 'misicuni', 'project', 'projects', 'railway', 'interconnection', 'project', 'enfe', 'governme

## AttnCut统计数据以及表征获取

### tf-idf

In [37]:
# 构建模型
from gensim import models

# tf-idf model
tfidf = models.TfidfModel(corpus_bow)

In [38]:
# tf-idf transformation
tfidf_dict = {}

for doc in tqdm(docset):
    text = docset[doc]['title'] + docset[doc]['abstractText']
    clean_text = get_corpus_element(text)
    tfidf_dict[doc] = tfidf[dictionary.doc2bow(clean_text)]

with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf_dict, f)

100%|██████████| 528155/528155 [04:52<00:00, 1808.30it/s]


In [39]:
tfidf_dict['FR940830-1-00100'][:3]

[(1, 0.13080508937535534),
 (29, 0.030722761357195563),
 (39, 0.027994690792143385)]

### doc2vec

In [40]:
# 文本清洗函数
def get_d2v_corpus(texts):
    clean = lambda t: re.sub('[,?;*!%^&_+():\[\]{}`~@#$=+\\|/<>.\'\"\d]', ' ', t.replace('"', ' ').replace('/', ' ').replace('\\', ' ').replace("'", ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('-', ' ').replace('.', '').replace('&hyph;', ' ').replace('&blank;', ' ').strip().lower())
    preprocessed_text = clean(texts)
    text_list = [word for word in preprocessed_text.lower().split() if (word not in STOPWORDS) and 1 < len(word) < 16]
    return text_list

In [41]:
train_corpus, tag = [], 0
for doc in tqdm(docset):
    text = docset[doc]['title'] + docset[doc]['abstractText']
    tokens = get_d2v_corpus(text)
    train_corpus.append(models.doc2vec.TaggedDocument(tokens, [tag]))
    tag += 1

100%|██████████| 528155/528155 [02:18<00:00, 3817.87it/s] 


In [46]:
model = models.doc2vec.Doc2Vec(vector_size=200, min_count=2, epochs=40)

In [47]:
model.build_vocab(train_corpus)

In [50]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2021-04-29 18:20:21,375 : WARNING : Effective 'alpha' higher than previous training cycles
2021-04-29 18:20:21,377 : INFO : training model with 3 workers on 347325 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2021-04-29 18:20:22,409 : INFO : EPOCH 1 - PROGRESS: at 0.32% examples, 406474 words/s, in_qsize 6, out_qsize 0
2021-04-29 18:20:23,428 : INFO : EPOCH 1 - PROGRESS: at 0.56% examples, 395965 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:20:24,482 : INFO : EPOCH 1 - PROGRESS: at 0.85% examples, 389735 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:20:25,521 : INFO : EPOCH 1 - PROGRESS: at 1.17% examples, 383157 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:20:26,545 : INFO : EPOCH 1 - PROGRESS: at 1.49% examples, 380233 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:20:27,547 : INFO : EPOCH 1 - PROGRESS: at 1.80% examples, 377000 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:20:28,558 : INFO : EPOCH 1 - PROGRESS: at 2.11% examples, 374152 words/s

2021-04-29 18:21:34,571 : INFO : EPOCH 1 - PROGRESS: at 22.45% examples, 405780 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:21:35,582 : INFO : EPOCH 1 - PROGRESS: at 22.75% examples, 405292 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:21:36,605 : INFO : EPOCH 1 - PROGRESS: at 23.08% examples, 404642 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:21:37,636 : INFO : EPOCH 1 - PROGRESS: at 23.47% examples, 403667 words/s, in_qsize 6, out_qsize 0
2021-04-29 18:21:38,655 : INFO : EPOCH 1 - PROGRESS: at 23.87% examples, 402922 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:21:39,663 : INFO : EPOCH 1 - PROGRESS: at 24.16% examples, 402331 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:21:40,682 : INFO : EPOCH 1 - PROGRESS: at 24.29% examples, 402455 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:21:41,704 : INFO : EPOCH 1 - PROGRESS: at 24.55% examples, 401977 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:21:42,721 : INFO : EPOCH 1 - PROGRESS: at 24.68% examples, 402125 words/s, in_qsiz

2021-04-29 18:22:48,720 : INFO : EPOCH 1 - PROGRESS: at 43.74% examples, 417385 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:49,736 : INFO : EPOCH 1 - PROGRESS: at 44.11% examples, 417289 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:50,738 : INFO : EPOCH 1 - PROGRESS: at 44.48% examples, 417160 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:51,748 : INFO : EPOCH 1 - PROGRESS: at 44.83% examples, 417078 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:52,757 : INFO : EPOCH 1 - PROGRESS: at 45.19% examples, 416938 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:53,797 : INFO : EPOCH 1 - PROGRESS: at 45.55% examples, 416775 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:54,810 : INFO : EPOCH 1 - PROGRESS: at 45.93% examples, 416700 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:55,823 : INFO : EPOCH 1 - PROGRESS: at 46.28% examples, 416559 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:22:56,844 : INFO : EPOCH 1 - PROGRESS: at 46.62% examples, 416452 words/s, in_qsiz

2021-04-29 18:24:03,043 : INFO : EPOCH 1 - PROGRESS: at 71.24% examples, 407754 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:24:04,068 : INFO : EPOCH 1 - PROGRESS: at 71.60% examples, 407544 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:24:05,080 : INFO : EPOCH 1 - PROGRESS: at 71.93% examples, 407271 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:24:06,090 : INFO : EPOCH 1 - PROGRESS: at 72.25% examples, 407059 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:24:07,103 : INFO : EPOCH 1 - PROGRESS: at 72.57% examples, 406791 words/s, in_qsize 6, out_qsize 0
2021-04-29 18:24:08,119 : INFO : EPOCH 1 - PROGRESS: at 72.89% examples, 406516 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:24:09,146 : INFO : EPOCH 1 - PROGRESS: at 73.22% examples, 406271 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:24:10,198 : INFO : EPOCH 1 - PROGRESS: at 73.61% examples, 406108 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:24:11,219 : INFO : EPOCH 1 - PROGRESS: at 74.01% examples, 406089 words/s, in_qsiz

2021-04-29 18:25:17,209 : INFO : EPOCH 1 - PROGRESS: at 94.84% examples, 407519 words/s, in_qsize 6, out_qsize 0
2021-04-29 18:25:18,246 : INFO : EPOCH 1 - PROGRESS: at 95.17% examples, 407563 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:25:19,286 : INFO : EPOCH 1 - PROGRESS: at 95.51% examples, 407609 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:25:20,292 : INFO : EPOCH 1 - PROGRESS: at 95.84% examples, 407660 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:25:21,295 : INFO : EPOCH 1 - PROGRESS: at 96.17% examples, 407719 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:25:22,320 : INFO : EPOCH 1 - PROGRESS: at 96.50% examples, 407741 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:25:23,329 : INFO : EPOCH 1 - PROGRESS: at 96.82% examples, 407791 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:25:24,347 : INFO : EPOCH 1 - PROGRESS: at 97.13% examples, 407823 words/s, in_qsize 5, out_qsize 0
2021-04-29 18:25:25,377 : INFO : EPOCH 1 - PROGRESS: at 97.46% examples, 407846 words/s, in_qsiz

...

2021-04-29 21:37:02,418 : INFO : EPOCH 39 - PROGRESS: at 97.17% examples, 425232 words/s, in_qsize 6, out_qsize 0
2021-04-29 21:37:03,420 : INFO : EPOCH 39 - PROGRESS: at 97.45% examples, 425035 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:37:04,432 : INFO : EPOCH 39 - PROGRESS: at 97.73% examples, 424890 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:37:05,433 : INFO : EPOCH 39 - PROGRESS: at 98.01% examples, 424689 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:37:06,473 : INFO : EPOCH 39 - PROGRESS: at 98.30% examples, 424477 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:37:07,502 : INFO : EPOCH 39 - PROGRESS: at 98.58% examples, 424339 words/s, in_qsize 6, out_qsize 0
2021-04-29 21:37:08,513 : INFO : EPOCH 39 - PROGRESS: at 98.88% examples, 424160 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:37:09,524 : INFO : EPOCH 39 - PROGRESS: at 99.15% examples, 423986 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:37:10,535 : INFO : EPOCH 39 - PROGRESS: at 99.49% examples, 423981 words/s

2021-04-29 21:38:13,197 : INFO : EPOCH 40 - PROGRESS: at 19.82% examples, 423556 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:14,198 : INFO : EPOCH 40 - PROGRESS: at 20.12% examples, 424035 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:15,202 : INFO : EPOCH 40 - PROGRESS: at 20.37% examples, 424385 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:16,202 : INFO : EPOCH 40 - PROGRESS: at 20.56% examples, 425083 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:17,209 : INFO : EPOCH 40 - PROGRESS: at 20.85% examples, 425335 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:18,232 : INFO : EPOCH 40 - PROGRESS: at 21.17% examples, 425666 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:19,237 : INFO : EPOCH 40 - PROGRESS: at 21.50% examples, 425784 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:20,254 : INFO : EPOCH 40 - PROGRESS: at 21.86% examples, 425775 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:38:21,259 : INFO : EPOCH 40 - PROGRESS: at 22.20% examples, 425867 words/s

2021-04-29 21:39:26,185 : INFO : EPOCH 40 - PROGRESS: at 41.61% examples, 441907 words/s, in_qsize 6, out_qsize 0
2021-04-29 21:39:27,228 : INFO : EPOCH 40 - PROGRESS: at 41.99% examples, 441590 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:39:28,251 : INFO : EPOCH 40 - PROGRESS: at 42.36% examples, 441476 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:39:29,255 : INFO : EPOCH 40 - PROGRESS: at 42.74% examples, 441231 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:39:30,283 : INFO : EPOCH 40 - PROGRESS: at 43.13% examples, 441000 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:39:31,299 : INFO : EPOCH 40 - PROGRESS: at 43.54% examples, 440798 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:39:32,318 : INFO : EPOCH 40 - PROGRESS: at 43.94% examples, 440528 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:39:33,364 : INFO : EPOCH 40 - PROGRESS: at 44.34% examples, 440367 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:39:34,381 : INFO : EPOCH 40 - PROGRESS: at 44.71% examples, 440231 words/s

2021-04-29 21:40:39,596 : INFO : EPOCH 40 - PROGRESS: at 68.35% examples, 421661 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:40,610 : INFO : EPOCH 40 - PROGRESS: at 68.71% examples, 421380 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:41,625 : INFO : EPOCH 40 - PROGRESS: at 69.06% examples, 421060 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:42,645 : INFO : EPOCH 40 - PROGRESS: at 69.43% examples, 420739 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:43,721 : INFO : EPOCH 40 - PROGRESS: at 69.80% examples, 420395 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:44,744 : INFO : EPOCH 40 - PROGRESS: at 70.19% examples, 420119 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:45,756 : INFO : EPOCH 40 - PROGRESS: at 70.52% examples, 419863 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:46,772 : INFO : EPOCH 40 - PROGRESS: at 70.86% examples, 419560 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:40:47,778 : INFO : EPOCH 40 - PROGRESS: at 71.23% examples, 419276 words/s

2021-04-29 21:41:53,163 : INFO : EPOCH 40 - PROGRESS: at 91.30% examples, 412545 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:41:54,165 : INFO : EPOCH 40 - PROGRESS: at 91.59% examples, 412421 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:41:55,177 : INFO : EPOCH 40 - PROGRESS: at 91.85% examples, 412300 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:41:56,185 : INFO : EPOCH 40 - PROGRESS: at 92.11% examples, 412157 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:41:57,200 : INFO : EPOCH 40 - PROGRESS: at 92.41% examples, 412018 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:41:58,227 : INFO : EPOCH 40 - PROGRESS: at 92.71% examples, 411894 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:41:59,231 : INFO : EPOCH 40 - PROGRESS: at 93.00% examples, 411745 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:42:00,254 : INFO : EPOCH 40 - PROGRESS: at 93.29% examples, 411623 words/s, in_qsize 5, out_qsize 0
2021-04-29 21:42:01,268 : INFO : EPOCH 40 - PROGRESS: at 93.57% examples, 411475 words/s

In [52]:
test_vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(test_vector[:10])

[ 0.17300138  0.51965016  0.07172695  0.133567   -0.15887052 -0.12952779
  0.15274774  0.24226308 -0.10642537 -0.18588775]


In [53]:
doc2vec_dict = {}
for doc in tqdm(docset):
    text = docset[doc]['title'] + docset[doc]['abstractText']
    tokens = get_d2v_corpus(text)
    vector = model.infer_vector(tokens)
    doc2vec_dict[doc] = vector

with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/doc2vec.pkl', 'wb') as f:
    pickle.dump(doc2vec_dict, f)

100%|██████████| 528155/528155 [5:50:47<00:00, 25.09it/s]   


### 求相似度

In [3]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/ranked_list/bm25_ranked_list.json', 'r') as f:
    bm25_ranked_list = json.load(f)

In [3]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/ranked_list/drmm_ranked_list.json', 'r') as f:
    drmm_ranked_list = json.load(f)

In [3]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/tfidf.pkl', 'rb') as f:
    tfidf_dict = pickle.load(f)

In [4]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/doc2vec.pkl', 'rb') as f:
    doc2vec_dict = pickle.load(f)

In [5]:
from tqdm import tqdm

In [6]:
def sparse2indice(vec):
    index = [item[0] for item in vec]
    value = [item[1] for item in vec]
    return [index, value], max(index) if index else 0

def rl_tfidf(ranked_list, tfidf_dict):
    rl_si, max_index = {}, 0
    for query in tqdm(ranked_list):
        rl_si[query] = []
        for doc in ranked_list[query]:
            i_v, max_d = sparse2indice(tfidf_dict[doc])
            rl_si[query].append(i_v)
            max_index = max(max_index, max_d)
    return rl_si, max_index

In [8]:
bm25_si, bm25_max_index = rl_tfidf(bm25_ranked_list, tfidf_dict)

100%|██████████| 243/243 [00:01<00:00, 127.59it/s]


In [7]:
drmm_si, drmm_max_index = rl_tfidf(drmm_ranked_list, tfidf_dict)

100%|██████████| 243/243 [00:02<00:00, 119.30it/s]


In [8]:
def iv_dense(iv, total):
    dense = [0] * total
    for i in range(len(iv[0])):
        dense[iv[0][i]] = iv[1][i]
    return dense

def rl_dense(rl_si):
    rl_dense = {}
    for query in tqdm(rl_si):
        rl_dense[query] = []
        for iv in rl_si[query]:
            rl_dense[query].append(iv_dense(iv, 231448))
    return rl_dense

In [10]:
bm25_dense = rl_dense(bm25_si)

100%|██████████| 243/243 [03:35<00:00,  1.13it/s]


In [11]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bm25_rl_tfidf.pkl', 'wb') as f:
    pickle.dump(bm25_dense, f)

In [9]:
drmm_dense = rl_dense(drmm_si)

100%|██████████| 243/243 [03:36<00:00,  1.12it/s]


In [10]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/drmm_rl_tfidf.pkl', 'wb') as f:
    pickle.dump(drmm_dense, f)

In [2]:
import numpy as np
def cos_similarity(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    sim = (num / denom) if denom != 0 else 0
    return sim if not np.isnan(sim) else 0

In [3]:
def neighbor_sim(vec_list, key):
    sim_list = [cos_similarity(np.array(vec_list[0]), np.array(vec_list[1]))]
    for i in tqdm(range(1, len(vec_list)-1), desc='calculate for qid_{}:'.format(key)):
        sim_list.append((cos_similarity(np.array(vec_list[i]), np.array(vec_list[i-1])) + cos_similarity(np.array(vec_list[i]), np.array(vec_list[i+1]))) / 2)
    sim_list.append(cos_similarity(np.array(vec_list[-2]), np.array(vec_list[-1])))
    return sim_list

In [4]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/drmm_rl_tfidf.pkl', 'rb') as f:
    drmm_dense = pickle.load(f)

In [5]:
drmm_sim = {}
for key in drmm_dense:
    drmm_sim[key] = neighbor_sim(drmm_dense[key], key)

calculate for qid_677:: 100%|██████████| 298/298 [00:44<00:00,  6.63it/s]


calculate for qid_700:: 100%|██████████| 298/298 [00:23<00:00, 12.63it/s]


In [6]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/drmm_tfidf_sim.pkl', 'wb') as f:
    pickle.dump(drmm_sim, f)

In [4]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bm25_rl_tfidf.pkl', 'rb') as f:
    bm25_dense = pickle.load(f)

In [8]:
bm25_sim = {}
for key in bm25_dense:
    bm25_sim[key] = neighbor_sim(bm25_dense[key], key)

calculate for qid_432:: 100%|██████████| 298/298 [00:43<00:00,  6.86it/s]


calculate for qid_698:: 100%|██████████| 298/298 [00:43<00:00,  6.84it/s]


In [9]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bm25_tfidf_sim.pkl', 'wb') as f:
    pickle.dump(bm25_sim, f)

In [7]:
bm25_d2v = {}
for query in tqdm(bm25_ranked_list):
    bm25_d2v[query] = []
    for doc in bm25_ranked_list[query]:
        bm25_d2v[query].append(doc2vec_dict[doc])

100%|██████████| 243/243 [00:00<00:00, 4522.73it/s]


In [8]:
bm25_d2v_sim = {}
for key in tqdm(bm25_d2v):
    bm25_d2v_sim[key] = neighbor_sim(bm25_d2v[key], key)

calculate for qid_301:: 100%|██████████| 298/298 [00:00<00:00, 18857.06it/s]

calculate for qid_303:: 100%|██████████| 298/298 [00:00<00:00, 19441.94it/s]

calculate for qid_306:: 100%|██████████| 298/298 [00:00<00:00, 17618.90it/s]

calculate for qid_310:: 100%|██████████| 298/298 [00:00<00:00, 19671.42it/s]

calculate for qid_315:: 100%|██████████| 298/298 [00:00<00:00, 17283.90it/s]

calculate for qid_321:: 100%|██████████| 298/298 [00:00<00:00, 17886.93it/s]

calculate for qid_325:: 100%|██████████| 298/298 [00:00<00:00, 16785.33it/s]

calculate for qid_327:: 100%|██████████| 298/298 [00:00<00:00, 17765.40it/s]

calculate for qid_331:: 100%|██████████| 298/298 [00:00<00:00, 15484.81it/s]

calculate for qid_332:: 100%|██████████| 298/298 [00:00<00:00, 18169.83it/s]

calculate for qid_333:: 100%|██████████| 298/298 [00:00<00:00, 15577.63it/s]

calculate for qid_334:: 100%|██████████| 298/298 [00:00<00:00, 18576.52it/s]

calculate for qid_336:: 100%|██████████| 298/298 [00:00<00:00, 1

calculate for qid_676:: 100%|██████████| 298/298 [00:00<00:00, 18135.56it/s]

calculate for qid_677:: 100%|██████████| 298/298 [00:00<00:00, 18550.60it/s]

calculate for qid_678:: 100%|██████████| 298/298 [00:00<00:00, 18970.69it/s]

calculate for qid_696:: 100%|██████████| 298/298 [00:00<00:00, 18243.29it/s]

calculate for qid_699:: 100%|██████████| 298/298 [00:00<00:00, 19043.23it/s]

calculate for qid_302:: 100%|██████████| 298/298 [00:00<00:00, 18589.78it/s]

calculate for qid_304:: 100%|██████████| 298/298 [00:00<00:00, 19198.26it/s]

calculate for qid_305:: 100%|██████████| 298/298 [00:00<00:00, 18855.07it/s]

calculate for qid_309:: 100%|██████████| 298/298 [00:00<00:00, 18638.85it/s]

calculate for qid_317:: 100%|██████████| 298/298 [00:00<00:00, 18467.01it/s]

calculate for qid_318:: 100%|██████████| 298/298 [00:00<00:00, 18194.43it/s]

calculate for qid_322:: 100%|██████████| 298/298 [00:00<00:00, 18842.00it/s]

calculate for qid_323:: 100%|██████████| 298/298 [00:00<00:00, 1

In [10]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bm25_d2v_sim.pkl', 'wb') as f:
    pickle.dump(bm25_d2v_sim, f)

In [11]:
drmm_d2v = {}
for query in tqdm(drmm_ranked_list):
    drmm_d2v[query] = []
    for doc in drmm_ranked_list[query]:
        drmm_d2v[query].append(doc2vec_dict[doc])

100%|██████████| 243/243 [00:00<00:00, 3773.25it/s]


In [12]:
drmm_d2v_sim = {}
for key in tqdm(drmm_d2v):
    drmm_d2v_sim[key] = neighbor_sim(drmm_d2v[key], key)

calculate for qid_301:: 100%|██████████| 298/298 [00:00<00:00, 15042.39it/s]

calculate for qid_302:: 100%|██████████| 298/298 [00:00<00:00, 17203.73it/s]

calculate for qid_303:: 100%|██████████| 298/298 [00:00<00:00, 17921.29it/s]

calculate for qid_304:: 100%|██████████| 298/298 [00:00<00:00, 17638.55it/s]

calculate for qid_306:: 100%|██████████| 298/298 [00:00<00:00, 18094.08it/s]

calculate for qid_307:: 100%|██████████| 298/298 [00:00<00:00, 19672.97it/s]

calculate for qid_308:: 100%|██████████| 298/298 [00:00<00:00, 19478.60it/s]

calculate for qid_309:: 100%|██████████| 298/298 [00:00<00:00, 18792.98it/s]

calculate for qid_310:: 100%|██████████| 298/298 [00:00<00:00, 18443.30it/s]

calculate for qid_313:: 100%|██████████| 298/298 [00:00<00:00, 19527.29it/s]

calculate for qid_314:: 100%|██████████| 298/298 [00:00<00:00, 18992.59it/s]

calculate for qid_315:: 100%|██████████| 298/298 [00:00<00:00, 19610.62it/s]

calculate for qid_317:: 100%|██████████| 298/298 [00:00<00:00, 1

calculate for qid_647:: 100%|██████████| 298/298 [00:00<00:00, 20350.09it/s]

calculate for qid_648:: 100%|██████████| 298/298 [00:00<00:00, 20073.27it/s]

calculate for qid_649:: 100%|██████████| 298/298 [00:00<00:00, 20635.67it/s]

calculate for qid_650:: 100%|██████████| 298/298 [00:00<00:00, 20590.12it/s]

calculate for qid_651:: 100%|██████████| 298/298 [00:00<00:00, 20081.98it/s]

calculate for qid_653:: 100%|██████████| 298/298 [00:00<00:00, 19409.03it/s]

calculate for qid_654:: 100%|██████████| 298/298 [00:00<00:00, 19887.71it/s]

calculate for qid_655:: 100%|██████████| 298/298 [00:00<00:00, 20137.31it/s]

calculate for qid_656:: 100%|██████████| 298/298 [00:00<00:00, 20093.60it/s]

calculate for qid_657:: 100%|██████████| 298/298 [00:00<00:00, 20327.26it/s]

calculate for qid_659:: 100%|██████████| 298/298 [00:00<00:00, 19887.39it/s]

calculate for qid_660:: 100%|██████████| 298/298 [00:00<00:00, 21523.70it/s]

calculate for qid_661:: 100%|██████████| 298/298 [00:00<00:00, 2

In [14]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/drmm_d2v_sim.pkl', 'wb') as f:
    pickle.dump(drmm_d2v_sim, f)

## Bicut输入数据

In [2]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bicut_stats.pkl', 'rb') as f:
    bicut_stats = pickle.load(f)

In [3]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/ranked_list/bm25_ranked_list.json', 'r') as f:
    bm25_ranked_list = json.load(f)

In [3]:
def iv_dense(iv, total):
    dense = [0] * total
    for item in iv:
        dense[item[0]] = item[1]
    return dense

In [6]:
bicut_bm25 = {}
for query in tqdm(bm25_ranked_list):
    bicut_bm25[query] = []
    for doc in bm25_ranked_list[query]:
        stats = bicut_stats[doc]
        temp_vec = stats[:2]
        temp_vec.extend(iv_dense(stats[2], 231448))
        bicut_bm25[query].append(temp_vec)

100%|██████████| 243/243 [05:12<00:00,  1.29s/it]


In [7]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bicut_bm25_input.pkl', 'wb') as f:
    pickle.dump(bicut_bm25, f)

In [4]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/ranked_list/drmm_ranked_list.json', 'r') as f:
    drmm_ranked_list = json.load(f)

In [5]:
bicut_drmm = {}
for query in tqdm(drmm_ranked_list):
    bicut_drmm[query] = []
    for doc in drmm_ranked_list[query]:
        stats = bicut_stats[doc]
        temp_vec = stats[:2]
        temp_vec.extend(iv_dense(stats[2], 231448))
        bicut_drmm[query].append(temp_vec)

100%|██████████| 243/243 [05:23<00:00,  1.33s/it]


In [6]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bicut_drmm_input.pkl', 'wb') as f:
    pickle.dump(bicut_drmm, f)

## AttnCut输入数据

In [2]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bicut_stats.pkl', 'rb') as f:
    bicut_stats = pickle.load(f)

In [3]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/ranked_list/bm25_ranked_list.json', 'r') as f:
    bm25_ranked_list = json.load(f)

In [4]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bm25_tfidf_sim.pkl', 'rb') as f:
    bm25_tfidf_sim = pickle.load(f)

In [5]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bm25_d2v_sim.pkl', 'rb') as f:
    bm25_d2v_sim = pickle.load(f)

In [6]:
attncut_bm25 = {}
for query in tqdm(bm25_ranked_list):
    attncut_bm25[query] = []
    for index, doc in enumerate(bm25_ranked_list[query]):
        stats = bicut_stats[doc]
        temp_vec = stats[:2]
        temp_vec.append(bm25_tfidf_sim[query][index])
        temp_vec.append(bm25_d2v_sim[query][index])
        attncut_bm25[query].append(temp_vec)

100%|██████████| 243/243 [00:01<00:00, 237.70it/s]


In [7]:
attncut_bm25['301'][0]

[105, 34, 0.3122440280135308, 0.30473042]

In [8]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/attncut_bm25_input.pkl', 'wb') as f:
    pickle.dump(attncut_bm25, f)

In [9]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/ranked_list/drmm_ranked_list.json', 'r') as f:
    drmm_ranked_list = json.load(f)

In [10]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/drmm_tfidf_sim.pkl', 'rb') as f:
    drmm_tfidf_sim = pickle.load(f)

In [11]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/drmm_d2v_sim.pkl', 'rb') as f:
    drmm_d2v_sim = pickle.load(f)

In [12]:
attncut_drmm = {}
for query in tqdm(drmm_ranked_list):
    attncut_drmm[query] = []
    for index, doc in enumerate(drmm_ranked_list[query]):
        stats = bicut_stats[doc]
        temp_vec = stats[:2]
        temp_vec.append(drmm_tfidf_sim[query][index])
        temp_vec.append(drmm_d2v_sim[query][index])
        attncut_drmm[query].append(temp_vec)

100%|██████████| 243/243 [00:00<00:00, 1794.18it/s]


In [13]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/attncut_drmm_input.pkl', 'wb') as f:
    pickle.dump(attncut_drmm, f)

### 校验数据一致性

#### 校验数据集分割前后的的数据一致性

In [2]:
import pickle

In [21]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/BM25_results/split_1/BM25_train_s1.pkl', 'rb') as f:
    raw = pickle.load(f)

In [22]:
raw.keys()

dict_keys(['368', '307', '429', '339', '305', '338', '414', '365', '669', '318', '654', '664', '611', '625', '419', '400', '357', '679', '325', '647', '636', '655', '436', '424', '432', '301', '304', '666', '433', '347', '657', '411', '443', '363', '607', '328', '321', '331', '696', '379', '310', '646', '389', '416', '675', '622', '334', '407', '423', '689', '352', '630', '434', '641', '435', '610', '426', '337', '614', '332', '690', '674', '450', '375', '408', '449', '627', '384', '649', '632', '329', '651', '412', '637', '624', '425', '662', '673', '700', '616', '341', '639', '602', '342', '371', '406', '687', '439', '694', '373', '618', '326', '383', '405', '420', '377', '319', '324', '336', '309', '353', '660', '422', '421', '427', '633', '613', '315', '447', '629', '672', '698', '346', '386', '608', '653', '320', '659', '685', '695', '626', '303', '446', '308', '621', '370', '684', '676', '648', '391', '367', '665', '330', '682', '392', '620', '350', '656', '448', '343', '314', '3

In [23]:
a = []
for doc in raw['368']['retrieved_documents']:
    a.append(doc['doc_id']) 

In [24]:
import json
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/ranked_list/bm25_ranked_list.json', 'r') as f:
    bm25_ranked_list = json.load(f)

In [25]:
bm25_ranked_list['368'][:10]

['LA120789-0015',
 'LA062389-0034',
 'LA032790-0011',
 'LA092189-0226',
 'LA080789-0101',
 'LA092689-0055',
 'LA080790-0131',
 'LA072390-0045',
 'LA041390-0072',
 'LA101689-0053']

In [26]:
a[:10]

['LA120789-0015',
 'LA062389-0034',
 'LA032790-0011',
 'LA092189-0226',
 'LA080789-0101',
 'LA092689-0055',
 'LA080790-0131',
 'LA072390-0045',
 'LA041390-0072',
 'LA101689-0053']

#### 校验拼接前后的数据一致性

In [1]:
import pickle

In [1]:
BM25_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/BM25_results'
DRMM_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/drmm_results'
DRMM_TKS_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/drmm_tks_results'
STATS_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics'
GT_PATH = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/robust04_data/robust04_gt.pkl'

In [4]:
split = 1
with open('{}/split_{}/BM25_train_s{}.pkl'.format(BM25_BASE, split, split), 'rb') as f:
    train_data_raw = pickle.load(f)

In [12]:
train_data_raw['368']['retrieved_documents'][:2]

[{'doc_id': 'LA120789-0015',
  'rank': 1,
  'bm25_score': 13.86861343,
  'norm_bm25_score': 4.723572638289525,
  'is_relevant': True},
 {'doc_id': 'LA062389-0034',
  'rank': 2,
  'bm25_score': 12.89498115,
  'norm_bm25_score': 4.2621051145931315,
  'is_relevant': True}]

In [13]:
scores = [train_data_raw['368']['retrieved_documents'][i]['norm_bm25_score'] for i in range(300)]

In [14]:
scores

[4.723572638289525,
 4.2621051145931315,
 3.8936928396960333,
 3.814352115937058,
 3.7917582234969043,
 3.6461272524997694,
 3.59123272339648,
 3.554855125999583,
 3.274706959381316,
 3.1763002487598517,
 3.028136319297676,
 2.8208909496215506,
 2.8101196565407607,
 2.7741817942963745,
 2.7410202857107917,
 2.686320399771917,
 2.576057793757048,
 2.312885215519527,
 2.1038913466800495,
 1.984434840680131,
 1.9540660565009078,
 1.7909048728793848,
 1.7787862587433494,
 1.7545094022662069,
 1.7263032725173337,
 1.6530516753278963,
 1.6449899210743228,
 1.6327372791446007,
 1.6327372791446007,
 1.5965197870900458,
 1.582927995259909,
 1.5703708735404471,
 1.5515637852635151,
 1.5449374337017203,
 1.5164738604395258,
 1.5118193734782819,
 1.4834968237316601,
 1.4680269459955724,
 1.4642142159524631,
 1.4110872962771899,
 1.3859761667876425,
 1.3843218681476483,
 1.3596084947396485,
 1.349764106370812,
 1.346585579277617,
 1.3374952216640554,
 1.291029210331246,
 1.2852500236270659,
 1.2485

In [15]:
docs = [train_data_raw['368']['retrieved_documents'][i]['doc_id'] for i in range(300)]

In [16]:
docs

['LA120789-0015',
 'LA062389-0034',
 'LA032790-0011',
 'LA092189-0226',
 'LA080789-0101',
 'LA092689-0055',
 'LA080790-0131',
 'LA072390-0045',
 'LA041390-0072',
 'LA101689-0053',
 'LA080889-0083',
 'LA081490-0123',
 'LA081289-0012',
 'LA111190-0025',
 'LA092790-0237',
 'LA052790-0032',
 'LA072490-0037',
 'LA081490-0032',
 'LA100189-0012',
 'LA090790-0018',
 'LA103090-0134',
 'FT934-1866',
 'FBIS4-34764',
 'FT921-4280',
 'LA102790-0172',
 'LA101689-0087',
 'LA102590-0184',
 'FT931-8242',
 'FT931-8327',
 'LA091589-0176',
 'FT934-11736',
 'FT934-9438',
 'LA102290-0115',
 'FBIS4-47541',
 'FT922-7131',
 'FR940603-2-00066',
 'FT932-11861',
 'LA092690-0088',
 'FT911-3467',
 'FR940817-2-00244',
 'FR940617-1-00076',
 'FR940617-1-00078',
 'LA102990-0096',
 'FBIS3-59716',
 'FR940817-2-00261',
 'FT943-15069',
 'FR940405-1-00139',
 'FR940617-1-00051',
 'LA030389-0179',
 'LA012989-0064',
 'FR940405-1-00146',
 'FR940114-2-00068',
 'LA091390-0249',
 'FR940525-2-00051',
 'FR940110-2-00064',
 'LA081289

In [17]:
with open('{}/attncut_bm25_input.pkl'.format(STATS_BASE), 'rb') as f:
    stats_bm25 = pickle.load(f)

In [20]:
stats_bm25['368'][:6]

[[58, 22, 0.11717447208653671, 0.42986637],
 [30, 14, 0.058587236043268355, 0.24583157896995544],
 [15, 7, 0.058273785520117964, 0.12449997663497925],
 [121, 38, 0.298266652331648, 0.35138192772865295],
 [20, 9, 0.3018175980517621, 0.4155770242214203],
 [128, 42, 0.0976967619024751, 0.3380442261695862]]

In [29]:
labels = [1 if train_data_raw['368']['retrieved_documents'][i]['is_relevant'] else 0 for i in range(300)]

In [2]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics/bicut_stats.pkl', 'rb') as f:
    bicut_stats = pickle.load(f)

In [5]:
bicut_stats['LA092689-0055'][:2]

[128, 42]